In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('filter_variable_list.csv')
df.head()

,Unnamed: 0,day_prob,state_prob,card_1_mean,card_1_max,card_1_median,card_1_sum,card_1_count,Merchnum_1_mean,Merchnum_1_max,...,Merchnum_sum_2_7,Merchnum_sum_2_14,Merchnum_sum_2_30,MerchDesc_sum_1_7,MerchDesc_sum_1_14,MerchDesc_sum_1_30,MerchDesc_sum_2_7,MerchDesc_sum_2_14,MerchDesc_sum_2_30,Fraud
0,3338,0.040022,0.000779,333.4700,333.47,333.47,333.47,1,161.160000,333.47,...,0.528995,0.040896,0.052622,0.528995,0.040896,0.052622,0.528995,0.040896,0.052622,0
1,3339,0.040022,0.008747,4.3092,7.30,4.37,107.73,25,5.387119,27.20,...,2.047618,0.096169,0.185860,6.503764,0.780509,2.185665,6.503764,0.780509,2.185665,0
2,3340,0.040022,0.002393,183.2950,315.66,206.26,733.18,4,242.726667,315.66,...,4.620473,0.143974,0.276900,4.620473,0.143974,0.276900,4.620473,0.143974,0.276900,0
3,3341,0.040022,0.008747,4.3092,7.30,4.37,107.73,25,5.387119,27.20,...,2.047618,0.096169,0.185860,6.503764,0.780509,2.185665,6.503764,0.780509,2.185665,0
4,3342,0.040022,0.000835,16.2800,16.28,16.28,16.28,1,16.280000,16.28,...,2.182306,0.153296,0.042862,2.182306,0.153296,0.042862,2.182306,0.153296,0.042862,0


In [4]:
df = df[df.columns[1:]]

In [12]:
df.dtypes

day_prob              float64
state_prob            float64
card_1_mean           float64
card_1_max            float64
card_1_median         float64
                       ...   
MerchDesc_sum_1_30    float64
MerchDesc_sum_2_7     float64
MerchDesc_sum_2_14    float64
MerchDesc_sum_2_30    float64
Fraud                   int64
Length: 424, dtype: object

In [14]:
#Then remove the first 2 weeks and the last 4 months (OOT)
len(df)

65708

In [64]:
df['random'] = np.random.normal(0, 1, size=(1, len(df)))[0]
df['random']

0        0.365110
1       -1.076690
2        0.664416
3        0.959994
4       -1.302899
           ...   
65703   -0.394960
65704   -0.619033
65705    0.167942
65706    0.306820
65707   -2.495550
Name: random, Length: 65708, dtype: float64

In [65]:
from scipy import stats as sps

goods = df[df.Fraud == 0]
bads = df[df.Fraud == 1]

KSFDR = pd.DataFrame({})

i = 0
for column in df:
    KSFDR.loc[i, 'ks'] = sps.ks_2samp(goods[column],bads[column])[0]
    i = i+1
    
KSFDR.index = df.columns

In [66]:
KSFDR

,ks
day_prob,0.142109
state_prob,0.378933
card_1_mean,0.546961
card_1_max,0.548025
card_1_median,0.516337
...,...
MerchDesc_sum_2_7,0.175545
MerchDesc_sum_2_14,0.149887
MerchDesc_sum_2_30,0.199739
Fraud,1.000000


In [67]:
topRows = int(round(len(df)*0.03))
j = 0
numbads = len(bads)

for column in df:
    temp = df.sort_values(column,ascending=False)
    temp1 = temp.head(topRows)
    temp2 = temp.tail(topRows)
    needed1 = temp1.loc[:,'Fraud']
    needed2 = temp2.loc[:,'Fraud']
    FDR1 = sum(needed1)/numbads
    FDR2 = sum(needed2)/numbads
    FDRate = np.maximum(FDR1,FDR2)
    KSFDR.loc[column, 'FDR'] = FDRate
    j = j +1


In [68]:
KSFDR

,ks,FDR
day_prob,0.142109,0.109986
state_prob,0.378933,0.095514
card_1_mean,0.546961,0.276411
card_1_max,0.548025,0.434153
card_1_median,0.516337,0.246020
...,...,...
MerchDesc_sum_2_7,0.175545,0.047757
MerchDesc_sum_2_14,0.149887,0.010130
MerchDesc_sum_2_30,0.199739,0.011577
Fraud,1.000000,1.000000


In [69]:
KSFDR['rank_ks'] = KSFDR['ks'].rank(ascending = True)
KSFDR['rank_FDR'] = KSFDR['FDR'].rank(ascending = True)
KSFDR['average_rank'] = (KSFDR['rank_ks'] + KSFDR['rank_FDR']) / 2
KSFDR.sort_values(by=['average_rank'], ascending=False, inplace=True)


In [70]:
KSFDR.loc['random']

ks               0.023244
FDR              0.027496
rank_ks          1.000000
rank_FDR        57.000000
average_rank    29.000000
Name: random, dtype: float64

In [139]:
KSFDR.reset_index().rename(columns={'index':'field'}).to_csv('finish-filter-list.csv', index = False)

In [134]:
#KSFDR.to_csv('finish-filter-list.csv')

In [82]:
def fdr_prob(y, y_prob, cutoff=0.03):
    if len(y_prob.shape) != 1:    # sometimes the proba list can contain many columns, one for each category
        y_prob = y_prob[:, -1:]   # only the last one (fraud_label==1) is used here.
    num_fraud = len(y[y == 1])    # count the total nunber of frauds   
# sort the proba list from high to low while retain the true (not predicted) fraud label
    sorted_prob = np.asarray(sorted(zip(y_prob, y), key=lambda x: x[0], reverse=True))
    cutoff_bin = sorted_prob[0:int(len(y) * cutoff), 1:]  # 3% cutoff
# return the FDR score (#fraud_in_cutoff / #total_fraud)
    return len(cutoff_bin[cutoff_bin == 1]) / num_fraud 


def fdr(classifier, x, y, cutoff=0.03):
# Calculates FDR score for the given classifier on dataset x and y with cutoff value
# get the probability list from the given classifier
    return fdr_prob(y, classifier.predict_proba(x), cutoff)

In [103]:
top_80 = KSFDR.reset_index().iloc[:81]['index'].to_list()[1:]

data = df[top_80]

Y = df['Fraud'].values

In [85]:
pip install mlxtend  

     |████████████████████████████████| 1.3 MB 16.7 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [104]:
#wrapper
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.ensemble import RandomForestClassifier

nfeatures = len(data.columns)
clf = RandomForestClassifier(n_estimators=5, n_jobs=-1)
sfs = SFS(clf,k_features=nfeatures,forward=True,verbose=2,scoring=fdr,cv=2,n_jobs=-1)
# sfs = SFS(clf,k_features=1,forward=False,verbose=2,scoring=fdr,cv=2,n_jobs=-1)
sfs.fit(data,Y)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:   16.9s finished

[2021-05-18 00:46:15] Features: 1/80 -- score: 0.3660844433274692[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done  79 out of  79 | elapsed:   15.2s finished

[2021-05-18 00:46:30] Features: 2/80 -- score: 0.5354821144341124[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  78 out of  78 | elapsed:   13.9s finished

[2021-05-18 00:46:45] Features: 3/80 -- score: 0.6122350674373795[Parallel(n_jobs=-1)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  77 out o

SequentialFeatureSelector(cv=2,
                          estimator=RandomForestClassifier(n_estimators=5,
                                                           n_jobs=-1),
                          k_features=80, n_jobs=-1,
                          scoring=<function fdr at 0x7f9f2be1b790>, verbose=2)

In [106]:
vars_FS = pd.DataFrame.from_dict(sfs.get_metric_dict()).T
vars_30 = list(vars_FS.loc[30, 'feature_names'])
vars_FS.to_csv('vars_FS.csv')

In [121]:
importance = []
for i in range(30):
    for var in vars_FS.loc[i+1,'feature_names']:
        if var not in importance:
            importance.append(var)
            


In [133]:
top_30 = pd.DataFrame(importance, columns=['field'])
top_30.to_csv('top_30_wrapper.csv', index = False)